In [59]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import linear_kernel

In [60]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [61]:
credits=pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
movies=pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')

In [62]:
credits.columns

In [64]:
movies.columns

In [66]:
credits.columns = ['id','title','cast','crew'] 
movies= movies.merge(credits,on='id')

In [ ]:
movies['overview'].head(5)

In [ ]:
movies['overview'] = movies['overview'].fillna('')

In [ ]:
def create_soup(x): 
    return''.join(x['keywords'])+''+''.join(x['genres'])+''+''.join(x['overview'])
movies['soup'] = movies.apply(create_soup, axis=1)

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
tfldf_matrix = tfidf.fit_transform(movies['soup']) 
tfidf_matrix.shape

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
#Construct a reverse up of indices and movie titles 
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()

In [ ]:
def get_recommendations(title,cosine_sim=cosine_sim): 
#Get the index of the movies that matches the title 
 idx = indices[title]

# Get the pairwise similarity scores of all movies with that movie 
 sim_scores = list(enumerate(cosine_sim[idx]))

#Sort the movies based on the similarity scores 
 sim_scores= sorted(sim_scores, key=lambda x: x[1],reverse=True)

#Get the scores of the 10 most similar movies 
 sim_scores = sim_scores[1:11]

# Get the movie indices
 movie_indices = [i[0] for i in sim_scores]

# Return the top 10 most similar movies 
 return movies['title'].iloc[movie_indices]